# Goodreads-Books EDA



In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'goodreadsbooks:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F231310%2F996643%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240518%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240518T103637Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2b3bdd3ce86dccb328cf4b90a9df56bed259296a8ad6fbbdcd8e0a6dd035926248e6d5debbe89ac297b1a6b480eb7ca62105e5b713cc1ea3ffa29f9810104ba7cd101c853fde45626ca7c6d139aed1bf139056ed4b748d5b44188fe6e15bb5612e3d35b0dc575edbdd924e5a3958171c1b8d72c68ce2f1b68a777f53a1e3ff7bfd50de43ae335442317a971de8294de825551a397d8bc8de5ba7f33efe2d7c39eb3e73c1aab283ed19daaae09f1ecd762d971bc372b3ceb0c51eef5c7654e102271d4920df9b82951b954f42695afdbb8eefdd261fce6d745490a3717b4b4e3d9e3356981eb342e974cfe3f7adaa5e9a9fe134dab9e26333802c5f4a9e248863'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 637338 bytes downloaded
Downloaded and uncompressed: goodreadsbooks
Data source import complete.


# General Information

## Loading Libraries & Data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/goodreadsbooks/books.csv", on_bad_lines="warn")
df

Skipping line 3350: expected 12 fields, saw 13
Skipping line 4704: expected 12 fields, saw 13
Skipping line 5879: expected 12 fields, saw 13
Skipping line 8981: expected 12 fields, saw 13



,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2/27/2007,Ecco


In [ ]:
df.head(5)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [ ]:
df.tail(5)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2/27/2007,Ecco
11122,45641,Las aventuras de Tom Sawyer,Mark Twain,3.91,8497646983,9788497646987,spa,272,113,12,5/28/2006,Edimat Libros


In [ ]:
df.head(15)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
5,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,0976540606,9780976540601,en-US,152,19,1,4/26/2005,Nimble Books
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,9780439827607,eng,3342,28242,808,9/12/2005,Scholastic
7,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,9780517226957,eng,815,3628,254,11/1/2005,Gramercy Books
8,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,Douglas Adams,4.38,0345453743,9780345453747,eng,815,249558,4080,4/30/2002,Del Rey Books
9,14,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams,4.22,1400052920,9781400052929,eng,215,4930,460,8/3/2004,Crown


In [ ]:
df.tail(15)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
11108,45595,La Tía Julia y el Escribidor,Mario Vargas Llosa,3.92,8432203238,9788432203237,spa,447,25,4,7/10/1977,Seix Barral
11109,45604,Narraciones Extraordinarias,Edgar Allan Poe,4.13,9583006408,9789583006401,spa,316,36,4,6/1/2004,Panamericana Editorial
11110,45607,Las Crónicas de Narnia,C.S. Lewis/Margarita E. Valdes/Gemma Gallart/P...,4.26,0061199001,9780061199004,spa,816,186,11,11/7/2006,HarperCollins Espanol
11111,45615,O Trono de Prata (As Crónicas de Nárnia #6),C.S. Lewis/Ana Falcão Bastos,3.96,972233168X,9789722331685,por,168,141,10,4/16/2004,Editorial Presença
11112,45616,A Última Batalha (As Crónicas de Nárnia #7),C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,4.03,9722332201,9789722332200,por,149,211,24,7/17/2004,Editorial Presença
11113,45617,O Cavalo e o Seu Rapaz (As Crónicas de Nárnia ...,C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,3.92,9722330551,9789722330558,por,160,207,16,8/15/2003,Editorial Presença
11114,45623,O Sobrinho do Mágico (As Crónicas de Nárnia #1),C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,4.04,9722329987,9789722329989,por,147,396,37,4/8/2003,Editorial Presença
11115,45625,A Viagem do Caminheiro da Alvorada (As Crónica...,C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,4.09,9722331329,9789722331326,por,176,161,14,9/1/2004,Editorial Presença
11116,45626,O Príncipe Caspian (As Crónicas de Nárnia #4),C.S. Lewis/Pauline Baynes/Ana Falcão Bastos,3.97,9722330977,9789722330978,por,160,215,11,10/11/2003,Editorial Presença
11117,45630,Whores for Gloria,William T. Vollmann,3.69,0140231579,9780140231571,en-US,160,932,111,2/1/1994,Penguin Books


In [ ]:
df.head(30)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
5,9,"Unauthorized Harry Potter Book Seven News: ""Ha...",W. Frederick Zimmerman,3.74,0976540606,9780976540601,en-US,152,19,1,4/26/2005,Nimble Books
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,0439827604,9780439827607,eng,3342,28242,808,9/12/2005,Scholastic
7,12,The Ultimate Hitchhiker's Guide: Five Complete...,Douglas Adams,4.38,0517226952,9780517226957,eng,815,3628,254,11/1/2005,Gramercy Books
8,13,The Ultimate Hitchhiker's Guide to the Galaxy ...,Douglas Adams,4.38,0345453743,9780345453747,eng,815,249558,4080,4/30/2002,Del Rey Books
9,14,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams,4.22,1400052920,9781400052929,eng,215,4930,460,8/3/2004,Crown


In [ ]:
df.tail(30)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
11093,45530,Montaillou: Cathars and Catholics in a French ...,Emmanuel Le Roy Ladurie/Barbara Bray,3.96,0140137009,9780140137002,eng,400,80,14,12/5/2002,Penguin
11094,45531,Montaillou village occitan de 1294 à 1324,Emmanuel Le Roy Ladurie/Emmanuel Le Roy-Ladurie,3.96,2070323285,9782070323289,fre,640,15,2,6/31/1982,Folio histoire
11095,45533,Montaillou: Cathars and Catholics in a French ...,Emmanuel Le Roy Ladurie/Barbara Bray,3.96,0140054715,9780140054712,en-GB,400,17,2,5/29/1980,Penguin Books Ltd.
11096,45536,On the Road,Jack Kerouac/Ann Charters,3.63,0142437255,9780142437254,en-US,307,3271,342,1/3/2006,Penguin Classics
11097,45546,Undaunted Courage: The Pioneering First Missio...,Stephen E. Ambrose,4.21,074347788X,9780743477888,eng,592,41236,1830,10/6/2003,Simon & Schuster
11098,45549,Undaunted Courage,Stephen E. Ambrose/Barrett Whitener,4.21,1415918090,9781415918098,eng,22,8,2,3/8/2005,Books on Tape
11099,45557,Co. Aytch: A Confederate Memoir of the Civil War,Sam R. Watkins,4.13,0684833247,9780684833248,eng,240,23,2,8/12/1997,Touchstone Books
11100,45564,Jonny Reb & Billy Yank,Alexander Hunter,4.11,1568520808,9781568520803,eng,635,14,2,5/31/1998,Not Avail
11101,45568,El Superzorro,Roald Dahl/Horacio Elena,4.05,9681907191,9789681907198,spa,96,174,20,6/1/2000,Alfaguara Infantil
11102,45570,Fantastic Mr Fox,David Wood/Roald Dahl,4.09,057305133X,9780573051333,en-GB,74,65,2,1/5/2011,Samuel French Ltd


## General Info

In [ ]:
df.shape

(11123, 12)

In [ ]:
df.columns

Index(['bookID', 'title', 'authors', 'average_rating', 'isbn', 'isbn13',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'],
      dtype='object')

In [ ]:
df.index

RangeIndex(start=0, stop=11123, step=1)

#### Basic Understanding of Dataset
We can observe from above that this dataset has 12 columns and 11123 rows of data. We can also observe in the initial data load that some rows (4 to be exact) had more fields than other rows so we skipped over them.  

# Info About Columns

In [ ]:
df.dtypes

bookID                  int64
title                  object
authors                object
average_rating        float64
isbn                   object
isbn13                  int64
language_code          object
  num_pages             int64
ratings_count           int64
text_reviews_count      int64
publication_date       object
publisher              object
dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11123 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11123 non-null  int64  
 1   title               11123 non-null  object 
 2   authors             11123 non-null  object 
 3   average_rating      11123 non-null  float64
 4   isbn                11123 non-null  object 
 5   isbn13              11123 non-null  int64  
 6   language_code       11123 non-null  object 
 7     num_pages         11123 non-null  int64  
 8   ratings_count       11123 non-null  int64  
 9   text_reviews_count  11123 non-null  int64  
 10  publication_date    11123 non-null  object 
 11  publisher           11123 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 1.0+ MB


#### Columns & Data Types:
* Here we have 12 columns, 1 of which is of **float64** data type, 5 are **int64** type and 6 are of **object** data type. So, we can summarize that we have 6 numerical columns and 6 object type/categorical columns.
* All of the columns are also have 11123 non-null values which means that we dont have missing values in columns



# Analysis of Numerical & Categorical Columns

In [ ]:
#filtering out numerical columns
df.select_dtypes(exclude="object")

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
0,1,4.57,9780439785969,652,2095690,27591
1,2,4.49,9780439358071,870,2153167,29221
2,4,4.42,9780439554893,352,6333,244
3,5,4.56,9780439655484,435,2339585,36325
4,8,4.78,9780439682589,2690,41428,164
...,...,...,...,...,...,...
11118,45631,4.06,9781560254416,512,156,20
11119,45633,4.08,9780140110876,635,783,56
11120,45634,3.96,9780140131963,415,820,95
11121,45639,3.72,9780060878825,434,769,139


In [ ]:
#getting insight about numerical columns
df.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11123.000000,11123.000000,1.112300e+04,11123.000000,1.112300e+04,11123.000000
mean,21310.856963,3.934075,9.759880e+12,336.405556,1.794285e+04,542.048099
std,13094.727252,0.350485,4.429758e+11,241.152626,1.124992e+05,2576.619589
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10277.500000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780582e+12,299.000000,7.450000e+02,47.000000
75%,32104.500000,4.140000,9.780872e+12,416.000000,5.000500e+03,238.000000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


In [ ]:
#filtering out object type columns
df.select_dtypes(include="object")

,title,authors,isbn,language_code,publication_date,publisher
0,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,0439785960,eng,9/16/2006,Scholastic Inc.
1,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,0439358078,eng,9/1/2004,Scholastic Inc.
2,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,0439554896,eng,11/1/2003,Scholastic
3,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,043965548X,eng,5/1/2004,Scholastic Inc.
4,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,0439682584,eng,9/13/2004,Scholastic
...,...,...,...,...,...,...
11118,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,1560254416,eng,12/21/2004,Da Capo Press
11119,You Bright and Risen Angels,William T. Vollmann,0140110879,eng,12/1/1988,Penguin Books
11120,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,0140131965,eng,8/1/1993,Penguin Books
11121,Poor People,William T. Vollmann,0060878827,eng,2/27/2007,Ecco


In [ ]:
#getting insight about object columns
df.describe(include="object")

,title,authors,isbn,language_code,publication_date,publisher
count,11123,11123,11123,11123,11123,11123
unique,10348,6639,11123,27,3679,2290
top,The Iliad,Stephen King,0439785960,eng,10/1/2005,Vintage
freq,9,40,1,8908,56,318


In [ ]:
cols_list = list(df.columns)
cols_list

['bookID',
 'title',
 'authors',
 'average_rating',
 'isbn',
 'isbn13',
 'language_code',
 '  num_pages',
 'ratings_count',
 'text_reviews_count',
 'publication_date',
 'publisher']

In [ ]:
#finding number of unique values for all object type columns
for col in cols_list:
    print(f"Unique Values for {col} are: {df[col].nunique()}")
    print("______________________________________________________")

Unique Values for bookID are: 11123
______________________________________________________
Unique Values for title are: 10348
______________________________________________________
Unique Values for authors are: 6639
______________________________________________________
Unique Values for average_rating are: 209
______________________________________________________
Unique Values for isbn are: 11123
______________________________________________________
Unique Values for isbn13 are: 11123
______________________________________________________
Unique Values for language_code are: 27
______________________________________________________
Unique Values for   num_pages are: 997
______________________________________________________
Unique Values for ratings_count are: 5294
______________________________________________________
Unique Values for text_reviews_count are: 1822
______________________________________________________
Unique Values for publication_date are: 3679
__________________

In [ ]:
df["language_code"].value_counts()

language_code
eng      8908
en-US    1408
spa       218
en-GB     214
fre       144
ger        99
jpn        46
mul        19
zho        14
grc        11
por        10
en-CA       7
ita         5
enm         3
lat         3
swe         2
rus         2
srp         1
nl          1
msa         1
glg         1
wel         1
ara         1
nor         1
tur         1
gla         1
ale         1
Name: count, dtype: int64

#### Some Insights about Data:
1. There seem to be no missing values as all columns have same number of rows i.e. 11123.
2. Since 'bookID', 'isbn' and 'isbn13' all have 11123 unique values, any one of these columns can serve as unique identifiers.
3. Out of all the object type columns, only 'language_code' has any chances of serving as a categorical column since the rest of columns have a large number of unique values.

##### Problems with 'language_code' column:
The language code column has 4 dialects of english as 4 separate categories i.e. eng, en-US, en-GB, and en-CA. These can all be renamed as eng to lower the number of categories.

# Cleaning Data

In [ ]:
# cleaning leading and trailing whitespaces

#finding cols of object data type
obj_cols = list(df.select_dtypes(include="object").columns)
print("Columns of obj data type:", obj_cols)

for col in obj_cols:
    df[col].str.strip()

print("cleaning complete")

Columns of obj data type: ['title', 'authors', 'isbn', 'language_code', 'publication_date', 'publisher']
cleaning complete


In [ ]:
# Replacing diff types of eng with generic eng language code in language_codes column
df['language_code'] = df['language_code'].str.replace("en-CA", "eng")
df['language_code'] = df['language_code'].str.replace("en-GB", "eng")
df['language_code'] = df['language_code'].str.replace("en-US", "eng")

df['language_code'].value_counts()

language_code
eng    10537
spa      218
fre      144
ger       99
jpn       46
mul       19
zho       14
grc       11
por       10
ita        5
lat        3
enm        3
rus        2
swe        2
nl         1
ara        1
srp        1
msa        1
glg        1
wel        1
nor        1
tur        1
gla        1
ale        1
Name: count, dtype: int64

# Data Exploration: loc & iLoc

In [ ]:
df.loc[10:20, ["title"]]

,title
10,The Hitchhiker's Guide to the Galaxy (Hitchhik...
11,The Ultimate Hitchhiker's Guide (Hitchhiker's ...
12,A Short History of Nearly Everything
13,Bill Bryson's African Diary
14,Bryson's Dictionary of Troublesome Words: A Wr...
15,In a Sunburned Country
16,I'm a Stranger Here Myself: Notes on Returning...
17,The Lost Continent: Travels in Small Town America
18,Neither Here nor There: Travels in Europe
19,Notes from a Small Island


In [ ]:
df.iloc[10:20, 1:3]

,title,authors
10,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Douglas Adams/Stephen Fry
11,The Ultimate Hitchhiker's Guide (Hitchhiker's ...,Douglas Adams
12,A Short History of Nearly Everything,Bill Bryson
13,Bill Bryson's African Diary,Bill Bryson
14,Bryson's Dictionary of Troublesome Words: A Wr...,Bill Bryson
15,In a Sunburned Country,Bill Bryson
16,I'm a Stranger Here Myself: Notes on Returning...,Bill Bryson
17,The Lost Continent: Travels in Small Town America,Bill Bryson
18,Neither Here nor There: Travels in Europe,Bill Bryson
19,Notes from a Small Island,Bill Bryson


In [ ]:
df.iloc[200:220, 1:4]

,title,authors,average_rating
200,The Red Notebook: True Stories,Paul Auster,3.77
201,Timbuktu / Leviathan / Moon Palace,Paul Auster,4.38
202,Collapse: How Societies Choose to Fail or Succeed,Jared Diamond,3.93
203,The Coming Economic Collapse: How You Can Thri...,Stephen Leeb/Glen C. Strathy,3.40
204,Collapse of Complex Societies,Joseph A. Tainter,4.15
205,Bowling Alone: The Collapse and Revival of Ame...,Robert D. Putnam,3.80
206,The Collapse of the Common Good: How America's...,Philip K. Howard,3.91
207,Reinventing the Enemy's Language: Contemporary...,Joy Harjo/Gloria Bird/Beth Cuthand/Valerie Mar...,4.39
208,My Inventions,Nikola Tesla,4.01
209,Wizard: The Life and Times of Nikola Tesla: Bi...,Marc J. Seifer/William H. Terbo,3.78


In [ ]:
df.loc[400:450, ['title','language_code', 'average_rating']]

,title,language_code,average_rating
400,The Histories,eng,3.99
401,The Histories,eng,3.99
402,The History,eng,3.99
403,The Iliad,eng,3.86
404,Iliad,eng,3.86
405,The Iliad,eng,3.86
406,The Iliad/The Odyssey,eng,4.04
407,The Iliad,eng,3.86
408,The Iliad,eng,3.86
409,The Essential Iliad,eng,3.86


# Finding Missing Values

In [ ]:
df.isnull()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,False,False,False,False,False,False,False,False,False,False,False,False
11119,False,False,False,False,False,False,False,False,False,False,False,False
11120,False,False,False,False,False,False,False,False,False,False,False,False
11121,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
df.isnull().sum()

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

#### Analysis:
We can see that none of the columns have any null or missing values

# Dealing with Duplicate Rows

In [ ]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
11118    False
11119    False
11120    False
11121    False
11122    False
Length: 11123, dtype: bool

In [ ]:
df.duplicated().sum()

0

It seems like there are no duplicate rows overall

# Conditions and Questions

#### Q: What are the titles that are being repeated

In [ ]:
duplicate_title_rows = df[df["title"].duplicated() == True]
duplicate_title_rows['title'].unique()

array(["The Hitchhiker's Guide to the Galaxy (Hitchhiker's Guide to the Galaxy  #1)",
       'The Lord of the Rings (The Lord of the Rings  #1-3)',
       'The Changeling', 'The Known World',
       'The Power of One (The Power of One  #1)', 'Anna Karenina',
       'The Portrait of a Lady', 'Treasure Island', "Gravity's Rainbow",
       'The Lovely Bones', 'Cien años de soledad', 'The Alchemist',
       'Memoirs of a Geisha', 'Deception Point',
       'The Richest Man in Babylon', '1776',
       'Freakonomics: A Rogue Economist Explores the Hidden Side of Everything',
       'Men Are from Mars  Women Are from Venus',
       "The Clan of the Cave Bear (Earth's Children  #1)", 'Phaedrus',
       'The Histories', 'The Iliad', 'The Odyssey', 'Hamlet', 'Medea',
       'The Oresteia', 'The Complete Plays', 'Lysistrata',
       'Metamorphoses', 'Antigone', 'Pride and Prejudice', 'Little Women',
       'A Tale of Two Cities',
       'Harry Potter and the Half-Blood Prince (Harry Potter  #6)',


#### Q: What are the repeating titles with the highest ratings

In [ ]:
df[(df["title"].duplicated() == True) & (df['average_rating']>4.5)]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
615,2005,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.57,0747584664,9780747584667,eng,768,1213,78,6/23/2006,Bloomsbury Publishing
1233,4256,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.56,074757362X,9780747573623,eng,480,3141,140,7/1/2008,Bloomsbury UK
4160,14905,The Complete Novels,Jane Austen,4.55,0140259449,9780140259445,eng,1344,21633,351,6/17/1996,Penguin
4263,15298,The Return of the King (The Lord of the Rings ...,J.R.R. Tolkien,4.53,0618574972,9780618574971,eng,508,1635,74,6/1/2005,Mariner Books
4277,15373,The Return of the King (The Lord of the Rings ...,J.R.R. Tolkien,4.53,0965307794,9780965307796,eng,440,202,10,1/1/2001,Quality Paperback Book Club
7512,28850,The Return of the King (The Lord of the Rings ...,J.R.R. Tolkien/Rob Inglis,4.53,0788789848,9780788789847,eng,16,3529,171,7/23/2001,Recorded Books Inc.
8239,31672,The Complete Novels,Jane Austen/Karen Joy Fowler,4.55,0143039504,9780143039501,eng,1278,425,39,3/28/2006,Penguin Classics
10346,41911,Harry Potter und der Gefangene von Askaban (Ha...,J.K. Rowling,4.56,355155210X,9783551552105,ger,448,30,2,4/1/2001,Carlsen


#### Q: What are Agatha Christie's highest rated books?

In [ ]:
books = df[(df['average_rating']>4) & (df['authors'] == 'Agatha Christie')]
books['title']

4518                             And Then There Were None
4521    Murder on the Orient Express (Hercule Poirot  ...
4526                                        Crooked House
4528       Witness for the Prosecution and Selected Plays
4529             The A.B.C. Murders (Hercule Poirot  #13)
4531     The Murder of Roger Ackroyd (Hercule Poirot  #4)
4534             Murder at the Vicarage (Miss Marple  #1)
4545    Murder on the Orient Express (Hercule Poirot  ...
4546              Death on the Nile (Hercule Poirot  #17)
4566      Hercule Poirot's Casebook (Hercule Poirot  #42)
4580          Poirot: The Complete Ariadne Oliver  Vol. 2
8146    Five Complete Miss Marple Novels: The Mirror C...
8150             Murder at the Vicarage (Miss Marple  #1)
8151    Miss Marple Omnibus Vol. 3 (Murder at the Vica...
Name: title, dtype: object

#### Q: Which books have the highest rating with most text reviews

In [ ]:
df[df['average_rating'] > 4.5].sort_values(by=['text_reviews_count'], ascending=False)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
4244,15195,The Complete Maus,Art Spiegelman,4.55,0141014083,9780141014081,eng,296,111475,5966,10/2/2003,Penguin Books
21,30,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,4.59,0345538374,9780345538376,eng,1728,101233,1550,9/25/2012,Ballantine Books
335,1111,The Power Broker: Robert Moses and the Fall of...,Robert A. Caro,4.51,0394720245,9780394720241,eng,1344,11208,1237,7/12/1975,Vintage
...,...,...,...,...,...,...,...,...,...,...,...,...
5648,20957,Colossians and Philemon: A Critical and Exeget...,R. McL. Wilson,5.00,0567044718,9780567044716,eng,512,1,0,12/7/2005,T&T Clark Int'l
5647,20954,Winchester Shotguns,Dennis Adler/R.L. Wilson,5.00,0785821082,9780785821083,eng,372,2,0,5/15/2008,Chartwell Books
5636,20913,The John Deere Two-Cylinder Tractor Encycloped...,Don Macmillan/Wayne G. Broehl Jr.,4.75,076032963X,9780760329634,eng,256,4,0,5/15/2007,Voyageur Press
5476,19788,The Goon Show Volume 11: He's Fallen in the W...,NOT A BOOK,5.00,0563388323,9780563388326,eng,2,2,0,10/2/1995,BBC Physical Audio


#### Q: Which books were published in the current century (after 2000) and have a 5 rating

---



In [ ]:
#converting date to a single format
df['publication_date'] = pd.to_datetime(df['publication_date'], format='%m/%d/%Y', errors="coerce")
df = df.dropna(subset=['publication_date'])
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11121 entries, 0 to 11122
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   bookID              11121 non-null  int64         
 1   title               11121 non-null  object        
 2   authors             11121 non-null  object        
 3   average_rating      11121 non-null  float64       
 4   isbn                11121 non-null  object        
 5   isbn13              11121 non-null  int64         
 6   language_code       11121 non-null  object        
 7     num_pages         11121 non-null  int64         
 8   ratings_count       11121 non-null  int64         
 9   text_reviews_count  11121 non-null  int64         
 10  publication_date    11121 non-null  datetime64[ns]
 11  publisher           11121 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(5)
memory usage: 1.1+ MB


In [ ]:
rows = df[(df['publication_date'] >= '2000-01-01') & (df['average_rating'] == 5)]
rows[['title', 'average_rating', 'publication_date']]

,title,average_rating,publication_date
786,Willem de Kooning: Late Paintings,5.0,2006-09-01
855,Literature Circle Guide: Bridge to Terabithia:...,5.0,2002-01-01
1243,Middlesex Borough (Images of America: New Jersey),5.0,2003-03-17
4125,Zone of the Enders: The 2nd Runner Official St...,5.0,2003-03-06
4788,The Diamond Color Meditation: Color Pathway to...,5.0,2006-02-01
5023,The Complete Theory Fun Factory: Music Theory ...,5.0,2004-06-01
5647,Winchester Shotguns,5.0,2008-05-15
5648,Colossians and Philemon: A Critical and Exeget...,5.0,2005-12-07
6247,The New Big Book of America,5.0,2002-03-21
8544,Fanning the Flame: Bible Cross and Mission,5.0,2003-06-17


#### Q: Which books were published in the current decade (after 2020) and have a 5 rating

In [ ]:
rows = df[(df['publication_date'] >= '2020-01-01') & (df['average_rating'] == 5)]
rows[['title', 'average_rating', 'publication_date']]

,title,average_rating,publication_date


# TASK 9: cut & qcut

In [ ]:
df['rating_category'] = pd.qcut(df['average_rating'], 3, labels=["Bad", "Average", "Good"])
df.head()

<ipython-input-46-5d22744a97e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rating_category'] = pd.qcut(df['average_rating'], 3, labels=["Bad", "Average", "Good"])


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,overall_rating,rating_category
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,Good,Good
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,Good,Good
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,Good,Good
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,Good,Good
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,Good,Good


In [ ]:
df['rating_category'].value_counts()

rating_category
Bad        3766
Average    3723
Good       3632
Name: count, dtype: int64

In [ ]:
df['overall_rating'] = pd.cut(df['average_rating'], 3, labels=["1", "2", "3"])
df.head()

<ipython-input-48-296d281e5ff3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overall_rating'] = pd.cut(df['average_rating'], 3, labels=["1", "2", "3"])


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,overall_rating,rating_category
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,2006-09-16,Scholastic Inc.,3,Good
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,2004-09-01,Scholastic Inc.,3,Good
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,2003-11-01,Scholastic,3,Good
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,2004-05-01,Scholastic Inc.,3,Good
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,2004-09-13,Scholastic,3,Good


In [ ]:
df['overall_rating'].value_counts()

overall_rating
3    10791
2      303
1       27
Name: count, dtype: int64

#### Difference Between Cut & QCut
* **qcut** made bins in a way that distributed the data evenly so we see an almost equal number of rows in each category.
* **cut** will however, make equally spaced intervals not caring about data distrbution so we saw that data was unevenly distributed among the three categories with the least number of rows in category 1.